In [1]:
# Import the required libraries

import pymongo
import datetime
import collections

import pandas as pd
import scipy.stats

import folium
import uuid

from bson.objectid import ObjectId
from bson.son import SON

In [2]:
# Open a connection to the Mongo server, open the accidents database and name the collections of accidents and labels
client = pymongo.MongoClient('mongodb://localhost:27351/')

db = client.accidents
accidents = db.accidents
labels = db.labels
roads = db.roads

In [3]:
# Load the expanded names of keys and human-readable codes into memory
expanded_name = collections.defaultdict(str)
for e in labels.find({'expanded': {"$exists": True}}):
    expanded_name[e['label']] = e['expanded']
    
label_of = collections.defaultdict(str)
for l in labels.find({'codes': {"$exists": True}}):
    for c in l['codes']:
        try:
            label_of[l['label'], int(c)] = l['codes'][c]
        except ValueError: 
            label_of[l['label'], c] = l['codes'][c]

# Activity 1

In [4]:
# Solution
def normalise_road_name(road_class, road_number):
    if road_number == 0:
        return None
    if road_class == 1:
        return 'M' + str(road_number)
    elif road_class == 2:
        if road_number == 1:
            return 'A1(M)'
        else:
            return 'A' + str(road_number) + 'M'
    elif road_class >= 3 and road_class <= 5:
        return label_of[('2nd_Road_Class', road_class)] + str(road_number)
    else:
        return None

In [5]:
# Test it works
[(normalise_road_name(a['1st_Road_Class'], a['1st_Road_Number']), 
  normalise_road_name(a['2nd_Road_Class'], a['2nd_Road_Number'])) 
 for a in accidents.find(limit=20)]

[('A308', None),
 ('B412', None),
 ('A3220', None),
 (None, None),
 ('B325', None),
 ('A308', 'A3220'),
 ('A3216', 'A4'),
 ('B450', None),
 (None, None),
 (None, None),
 ('A315', None),
 ('A315', 'A3220'),
 ('A402', 'A4206'),
 ('B415', 'B450'),
 ('B450', 'B412'),
 ('A3217', None),
 (None, None),
 ('A3220', 'A3220'),
 ('B316', None),
 ('A4204', None)]

# Activity 2

In [6]:
# Solution
def road_for_accident(accident)
    first_road_name = normalise_road_name(accident['1st_Road_Class'], accident['1st_Road_Number'])
    second_road_name = normalise_road_name(accident['2nd_Road_Class'], accident['2nd_Road_Number'])
    if first_road_name:
        first_road_result = db.command(SON([('geoNear', 'roads'), 
                ('near', accident['loc']),
                ('spherical', True),
                ('query', {'Road': first_road_name}),
                ('limit', 1)]))
    else:
        first_road_result = {'results': []}
    if second_road_name:
        second_road_result = db.command(SON([('geoNear', 'roads'), 
                ('near', accident['loc']),
                ('spherical', True),
                ('query', {'Road': second_road_name}),
                ('limit', 1)]))
    else:
        second_road_result = {'results': []}

    all_results = first_road_result['results'] + second_road_result['results']
    sorted_results = sorted(all_results, key=lambda r: r['dis'])
    
    if sorted_results:
        nearest_road = sorted_results[0]['obj']
        return nearest_road
    else:
        return None

In [7]:
# Test it works
road_for_accident(accidents.find_one())

{'A-Junction': 'A3220',
 'AADFYear': 2012,
 'B-Junction': 'A4',
 'CP': 16854,
 'Fd2WMV': 1083.0,
 'FdAll_MV': 17673.0,
 'FdBUS': 1114.0,
 'FdCar': 12867.0,
 'FdHGV': 687.0,
 'FdHGVA3': 2.0,
 'FdHGVA5': 13.0,
 'FdHGVA6': 3.0,
 'FdHGVR2': 492.0,
 'FdHGVR3': 109.0,
 'FdHGVR4': 67.0,
 'FdLGV': 1922.0,
 'FdPC': 1232.0,
 'Latitude': 51.491641149098996,
 'LenNet': 1.8,
 'LenNet_miles': 1.11,
 'Longitude': -0.17207541184187886,
 'ONS GOR Name': 'London',
 'ONS LA Name': 'Kensington and Chelsea',
 'RCat': 'PU',
 'Road': 'A308',
 'S Ref E': 527000,
 'S Ref N': 178550,
 '_id': ObjectId('533ed2c589f6f9ee18baf4d9'),
 'loc': {'coordinates': [-0.17207541184187886, 51.491641149098996],
  'type': 'Point'}}

In [ ]:
# Test it works a lot
res = []
for a in accidents.find(limit=20):
    rfa = road_for_accident(a)
    if rfa:
        res.append((rfa['Road'], rfa['CP'],
                    normalise_road_name(a['1st_Road_Class'], a['1st_Road_Number']),
                    normalise_road_name(a['2nd_Road_Class'], a['2nd_Road_Number'])))
    else:
        res.append((None, None,
                    normalise_road_name(a['1st_Road_Class'], a['1st_Road_Number']),
                    normalise_road_name(a['2nd_Road_Class'], a['2nd_Road_Number'])))
res

# Activity 3

In [ ]:
# Solution

# First, remove all the existing cached accident indexes
roads.update_many({}, {'$unset': {'nearby_accidents': True},
                  '$set': {'nearby_accident_count': 0}})

# Include the accident indexes in the road documents.
#   Also maintain a count of how many accidents there are for each record.
for a in accidents.find():
    rfa = road_for_accident(a)
    if rfa and ('nearby_accidents' not in rfa or a['Accident_Index'] not in rfa['nearby_accidents']):
        roads.update_one({'_id': rfa['_id']}, {'$push': {'nearby_accidents': a['Accident_Index']},
                                           '$inc': {'nearby_accident_count': 1}})

In [ ]:
list(roads.find({'CP': 16854}))